In [ ]:
import numpy as np
from boxx import *
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage
import numpy as np
import math
from matplotlib.colors import ListedColormap
from matplotlib.cm import hsv
from tqdm import tqdm
import cv2


def generate_colormap(number_of_distinct_colors: int = 80):
    if number_of_distinct_colors == 0:
        number_of_distinct_colors = 80

    number_of_shades = 7
    number_of_distinct_colors_with_multiply_of_shades = int(
        math.ceil(number_of_distinct_colors / number_of_shades) * number_of_shades
    )

    # Create an array with uniformly drawn floats taken from <0, 1) partition
    linearly_distributed_nums = (
        np.arange(number_of_distinct_colors_with_multiply_of_shades)
        / number_of_distinct_colors_with_multiply_of_shades
    )

    # We are going to reorganise monotonically growing numbers in such way that there will be single array with saw-like pattern
    #     but each saw tooth is slightly higher than the one before
    # First divide linearly_distributed_nums into number_of_shades sub-arrays containing linearly distributed numbers
    arr_by_shade_rows = linearly_distributed_nums.reshape(
        number_of_shades,
        number_of_distinct_colors_with_multiply_of_shades // number_of_shades,
    )

    # Transpose the above matrix (columns become rows) - as a result each row contains saw tooth with values slightly higher than row above
    arr_by_shade_columns = arr_by_shade_rows.T

    # Keep number of saw teeth for later
    number_of_partitions = arr_by_shade_columns.shape[0]

    # Flatten the above matrix - join each row into single array
    nums_distributed_like_rising_saw = arr_by_shade_columns.reshape(-1)

    # HSV colour map is cyclic (https://matplotlib.org/tutorials/colors/colormaps.html#cyclic), we'll use this property
    initial_cm = hsv(nums_distributed_like_rising_saw)

    lower_partitions_half = number_of_partitions // 2
    upper_partitions_half = number_of_partitions - lower_partitions_half

    # Modify lower half in such way that colours towards beginning of partition are darker
    # First colours are affected more, colours closer to the middle are affected less
    lower_half = lower_partitions_half * number_of_shades
    for i in range(3):
        initial_cm[0:lower_half, i] *= np.arange(0.2, 1, 0.8 / lower_half)

    # Modify second half in such way that colours towards end of partition are less intense and brighter
    # Colours closer to the middle are affected less, colours closer to the end are affected more
    for i in range(3):
        for j in range(upper_partitions_half):
            modifier = (
                np.ones(number_of_shades)
                - initial_cm[
                    lower_half
                    + j * number_of_shades : lower_half
                    + (j + 1) * number_of_shades,
                    i,
                ]
            )
            modifier = j * modifier / upper_partitions_half
            initial_cm[
                lower_half
                + j * number_of_shades : lower_half
                + (j + 1) * number_of_shades,
                i,
            ] += modifier

    return ListedColormap(initial_cm)


#!https://stackoverflow.com/questions/42697933/colormap-with-maximum-distinguishable-colours


N = 100
cmap = generate_colormap(N)
colors = cmap(np.linspace(0, 1, N))
colors = np.array(colors[:, :3]) * 255
colors = colors.astype(np.uint8).tolist()
colors.insert(0, [0, 0, 0])  # background is black, can be any color

# augmentations for the object detection task
# these are very light augmentations, because the models used for object detection seem to be very sensitive to changes
seq_json = iaa.Sequential(
    [
        iaa.CoarseDropout(0.001, size_percent=0.002),
        iaa.Crop(percent=(0, 0.002)),
        iaa.Sharpen((0, 0.5)),
        iaa.Sometimes(
            0.3,
            iaa.GaussianBlur(sigma=(0, 0.05)),
            iaa.AdditiveGaussianNoise(scale=(0, 2)),
        ),
        iaa.Affine(rotate=(-180, 180), shear=(-20, 20)),
        iaa.Sometimes(
            0.15,
            iaa.WithHueAndSaturation(iaa.WithChannels(0, iaa.Add((-7, 7)))),
            iaa.AddToBrightness((-10, 10)),
        ),
        #    iaa.Sometimes(
        #                0.05,
        #                iaa.ChangeColorTemperature((1000, 25000)),)
    ]
)

# augmentations for the segmentation task
seq_bin = iaa.Sequential(
    [
        iaa.CoarseDropout(0.001, size_percent=0.002),
        iaa.Crop(percent=(0, 0.002)),
        iaa.Sharpen((0, 0.5)),
        iaa.Sometimes(
            0.3,
            iaa.GaussianBlur(sigma=(0, 3)),
            iaa.AdditiveGaussianNoise(scale=(0, 10)),
        ),
        iaa.Sometimes(
            0.2,
            iaa.SomeOf((0, 2), [iaa.LinearContrast((0.5, 1.2)), iaa.Add((-10, 10))]),
        ),
        iaa.Affine(rotate=(-180, 180), shear=(-20, 20)),
        iaa.Sometimes(
            0.15,
            iaa.WithHueAndSaturation(iaa.WithChannels(0, iaa.Add((-20, 20)))),
            iaa.AddToBrightness((-20, 20)),
        ),
        iaa.Sometimes(
            0.05,
            iaa.ChangeColorTemperature((1000, 25000)),
        ),
    ]
)


# augmentation pipeline
def augment_crag(
    image_folder: str,
    annotation_folder: str,
    number_of_augmentations: int = 20,
    resize: bool = False,
    resize_shape: tuple = (256, 256),
    transforms=None,
    mode: str = "test",
):
    """
    image_folder: folder with images
    annotation_folder: folder with annotations
    number_of_augmentations: number of augmentations per image
    resize: resize images and annotations to resize_shape
    resize_shape: shape to resize to (makes the process faster)
    transforms: imgaug transforms
    mode: test or train (test does not augment)
    """
    ia.seed(1)
    assert mode in ["train", "test"]
    if mode == "test":
        for base_name in tqdm(os.listdir(image_folder)):
            if base_name.endswith(".png"):
                image_name = os.path.join(image_folder, base_name)
                annotation_name = os.path.join(annotation_folder, base_name)
                image = imageio.imread(image_name)
                annotation = imageio.imread(annotation_name)
                if resize:
                    image = cv2.resize(
                        image, resize_shape, interpolation=cv2.INTER_NEAREST
                    )
                    annotation = cv2.resize(
                        annotation, resize_shape, interpolation=cv2.INTER_NEAREST
                    )
                annotation = SegmentationMapsOnImage(annotation, shape=annotation.shape)
                imageio.imwrite(image_name, image)
                imageio.imwrite(annotation_name, annotation.draw(colors=colors)[0])
        return
    else:
        assert transforms is not None
        for base_name in tqdm(os.listdir(image_folder)):
            if base_name.endswith(".png"):
                image_name = os.path.join(image_folder, base_name)
                annotation_name = os.path.join(annotation_folder, base_name)
                image = imageio.imread(image_name)
                annotation = imageio.imread(annotation_name)
                if resize:
                    image = cv2.resize(
                        image, resize_shape, interpolation=cv2.INTER_NEAREST
                    )  # without interpolation change, the annotations get messed up
                    annotation = cv2.resize(
                        annotation, resize_shape, interpolation=cv2.INTER_NEAREST
                    )
                    imageio.imwrite(image_name, image)
                annotation = SegmentationMapsOnImage(annotation, shape=annotation.shape)
                for kind in range(number_of_augmentations):
                    new_base_name = base_name[:-4] + "_aug_" + str(kind) + ".png"
                    new_image_name = os.path.join(image_folder, new_base_name)
                    new_annotation_name = os.path.join(annotation_folder, new_base_name)
                    image_aug, annotation_aug = transforms(
                        image=image, segmentation_maps=annotation
                    )
                    imageio.imwrite(new_image_name, image_aug)
                    imageio.imwrite(
                        new_annotation_name, annotation_aug.draw(colors=colors)[0]
                    )
                # also put color on base annotation
                imageio.imwrite(annotation_name, annotation.draw(colors=colors)[0])


augment_crag(
    "/path/to/Images",
    "/path/to/Annotations",
    13,
    resize=True,
    resize_shape=(256, 256),
    mode="test",
    transforms=seq_bin,
)

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import glob

# https://github.com/XiaoyuZHK/CRAG-Dataset_Aug_ToCOCO
# @XiaoyuZHK https://github.com/XiaoyuZHK

# extract masks from the labels
def rgb2masks(label_name, local, bg_color=(0, 0, 0)):
    lbl_id = os.path.splitext(os.path.basename(label_name))[0]
    lbl = cv2.imread(label_name, cv2.IMREAD_COLOR)
    h, w = lbl.shape[:2]
    cell_dict = {}
    idx = 0
    for i in range(h):
        for j in range(w):
            color = tuple(lbl[i][j])
            if color in cell_dict or color == bg_color:
                continue
            cell_dict[color] = idx
            mask = (lbl == color).all(-1)
            mask_name = os.path.join(local, f"{lbl_id}_cell_{idx}.png")
            cv2.imwrite(
                mask_name, mask.astype(np.uint8) * 255
            )  # Save the mask directly without using white_mask
            idx += 1


label_dir = "/path/to/Annotations"
label_list = glob.glob(os.path.join(label_dir, "*.png"))

local = "/path/to/save_folder"
os.makedirs(local, exist_ok=True)

trainsum = 0
for label_name in tqdm(label_list):
    rgb2masks(label_name, local, bg_color=(0, 0, 0))
    trainsum += 1

print(trainsum)

In [ ]:
import os
import datetime

import pycococreatortools
import pycocotools
from PIL.Image import Image
from boxx import *
import cv2
import numpy as np
import os, glob
import json
from tqdm import tqdm
import os
import re
import fnmatch
from PIL import Image
import numpy as np


INFO = {
    "description": "Example Dataset",
    "url": "https://github.com/waspinator/pycococreator",
    "version": "0.1.0",
    "year": 2018,
    "contributor": "waspinator",
    "date_created": datetime.datetime.utcnow().isoformat(" "),
}

LICENSES = [
    {
        "id": 1,
        "name": "Attribution-NonCommercial-ShareAlike License",
        "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/",
    }
]

CATEGORIES = [
    {
        "id": 1,
        "name": "cell",
        "supercategory": "cell",
    },
]


def filter_for_jpeg(root, files):
    file_types = ["*.jpeg", "*.jpg", "*.png"]
    file_types = r"|".join([fnmatch.translate(x) for x in file_types])
    files = [os.path.join(root, f) for f in files]
    files = [f for f in files if re.match(file_types, f)]
    return files


def filter_for_annotations(root, files, image_filename):
    file_types = ["*.png"]
    file_types = r"|".join([fnmatch.translate(x) for x in file_types])
    basename_no_extension = os.path.splitext(os.path.basename(image_filename))[0]
    file_name_prefix = basename_no_extension + ".*"
    files = [os.path.join(root, f) for f in files]
    files = [f for f in files if re.match(file_types, f)]
    files = [
        f
        for f in files
        if re.match(file_name_prefix, os.path.splitext(os.path.basename(f))[0])
    ]
    return files


def translate(ROOT_DIR, IMAGE_DIR, ANNOTATION_DIR):
    coco_output = {
        "info": INFO,
        "licenses": LICENSES,
        "categories": CATEGORIES,
        "images": [],
        "annotations": [],
    }
    i = 0
    image_id = 1
    segmentation_id = 1

    # filter for jpeg images
    for root, _, files in tqdm(os.walk(IMAGE_DIR)):
        # 所有图片
        image_files = filter_for_jpeg(root, files)
        # tree-image_files
        finish = []
        # go through each image
        for image_filename in image_files:
            image = Image.open(image_filename)
            i += 1
            #             image_filename_json = image_filename.split('cell')

            #             image_filename_json = image_filename_json[0][:-1]+'.png'
            #             print(image_filename_json)
            # 创建image_info,这个其实就是每张图放进去而不是每个实例
            #             if image_filename_json not in finish:
            image_info = pycococreatortools.create_image_info(
                image_id, os.path.basename(image_filename), image.size
            )
            coco_output["images"].append(image_info)
            #             finish.append(image_filename_json)

            # filter for associated png annotations
            for root, _, files in os.walk(ANNOTATION_DIR):
                annotation_files = filter_for_annotations(root, files, image_filename)

                # go through each associated annotation
                for annotation_filename in annotation_files:
                    annotation_filename_json = annotation_filename.split("cell")

                    annotation_filename_json = annotation_filename_json[0][:-1] + ".png"

                    print(
                        "image_filename:",
                        image_filename,
                        "\nannotation_filename:",
                        annotation_filename,
                        "\n",
                        "annotation_filename_json:",
                        annotation_filename_json,
                        "\n",
                    )

                    image_filename_compare = image_filename.split("/")[-1]
                    annotation_filename_json_compare = annotation_filename_json.split(
                        "/"
                    )[-1]

                    if annotation_filename_json_compare == image_filename_compare:
                        print("!!!!!")
                        class_id = [
                            x["id"]
                            for x in CATEGORIES
                            if x["name"] in annotation_filename
                        ][0]

                        tree - class_id

                        category_info = {
                            "id": class_id,
                            "is_crowd": "crowd" in image_filename,
                        }
                        binary_mask = np.asarray(
                            Image.open(annotation_filename).convert("1")
                        ).astype(np.uint8)

                        annotation_info = pycococreatortools.create_annotation_info(
                            segmentation_id,
                            image_id,
                            category_info,
                            binary_mask,
                            image.size,
                            tolerance=2,
                        )

                        if annotation_info is not None:
                            coco_output["annotations"].append(annotation_info)

                        segmentation_id = segmentation_id + 1
            image_id = image_id + 1

    with open("{}/instances_test.json".format(ANNOTATION_DIR), "w") as output_json_file:
        json.dump(coco_output, output_json_file)
    print(i)


# create a .json file in coco format out of the annotations
def main():
    ROOT_DIR = "/path/to/main_folder"
    IMAGE_DIR = os.path.join(ROOT_DIR, "Images")
    ANNOTATION_DIR = os.path.join(ROOT_DIR, "Annotations_")

    translate(ROOT_DIR, IMAGE_DIR, ANNOTATION_DIR)

    translate(ROOT_DIR, IMAGE_DIR, ANNOTATION_DIR)


if __name__ == "__main__":
    main()

In [ ]:
import json
import cv2
import matplotlib.pyplot as plt
import os


# check data integrity by displaying images with annotations
def check_coco(
    instance_folder: str,
    image_folder: str,
    dataset: dict,
    annotations_style: str = "coco",
    amount: int = 10,
    show_agumetned_only: bool = False,
):
    assert annotations_style in ["coco", "circle"]
    check = 0
    print("Amount of images: ", len(dataset["images"]))
    if annotations_style == "coco":
        for image_info in dataset["images"]:
            if show_agumetned_only:
                if len(image_info["file_name"].split("_")) < 3:
                    continue

            print(image_info["file_name"])
            image = cv2.imread(os.path.join(image_folder, image_info["file_name"]))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            annotations = [
                ann
                for ann in dataset["annotations"]
                if ann["image_id"] == image_info["id"]
            ]

            for ann in annotations:
                bbox = ann["bbox"]
                x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 3)

            for instance in os.listdir(instance_folder):
                if instance.split(".")[0] == image_info["file_name"].split(".")[0]:
                    instance = cv2.imread(os.path.join(instance_folder, instance))
                    instance = cv2.cvtColor(instance, cv2.COLOR_BGR2RGB)
                    # brighten the instance to better evaluate the results
                    instance = instance * 255
                    plt.imshow(instance)
                    plt.show()

            plt.imshow(image)
            plt.show()
            check += 1
            if check == amount:
                break
    else:
        for image_info in dataset["images"]:
            print(image_info["file_name"])
            image = cv2.imread(os.path.join(image_folder, image_info["file_name"]))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            annotations = [
                ann
                for ann in dataset["annotations"]
                if ann["image_id"] == image_info["id"]
            ]

            for ann in annotations:
                x, y, r = (
                    ann["circle_center"][0],
                    ann["circle_center"][1],
                    ann["circle_radius"],
                )
                cv2.circle(image, (int(x), int(y)), int(r), (0, 255, 0), 2)

            for instance in os.listdir(instance_folder):
                if instance.split(".")[0] == image_info["file_name"].split(".")[0]:
                    instance = cv2.imread(os.path.join(instance_folder, instance))
                    instance = cv2.cvtColor(instance, cv2.COLOR_BGR2RGB)
                    # brighten the instance to better evaluate the results
                    instance = instance * 255
                    plt.imshow(instance)
                    plt.show()

            plt.imshow(image)
            plt.show()
            check += 1
            if check == amount:
                break


if __name__ == "__main__":
    image_folder = "/home/lorenz/JF/new_data/CRAG_v2/CRAG/test/Images"
    instance_folder = "/home/lorenz/JF/new_data/CRAG_v2/CRAG/test/Annotation"
    with open(
        "/home/lorenz/JF/new_data/CRAG_v2/CRAG/test/JSON/instances_test.json", "r"
    ) as f:
        dataset = json.load(f)

    check_coco(
        instance_folder,
        image_folder,
        dataset,
        annotations_style="coco",
        amount=10,
        show_agumetned_only=False,
    )